In [1]:
import struct
import numpy as np

def load_idx_images(filename):
    with open(filename, 'rb') as f:
        magic, num, rows, cols = struct.unpack('>IIII', f.read(16))
        data = np.frombuffer(f.read(), dtype=np.uint8).reshape(num, rows * cols)
        data = data.astype(np.float32) / 255.0
        return data

def load_idx_labels(filename):
    with open(filename, 'rb') as f:
        magic, num = struct.unpack('>II', f.read(8))
        labels = np.frombuffer(f.read(), dtype=np.uint8)
        return labels

In [2]:
def relu(x):
    return np.maximum(0, x)

def relu_deriv(x):
    return (x > 0).astype(float)

def softmax(x):
    x -= np.max(x, axis=1, keepdims=True)  # stability
    e = np.exp(x)
    return e / np.sum(e, axis=1, keepdims=True)

In [3]:
def cross_entropy(pred, labels):
    n = labels.shape[0]
    return -np.sum(np.log(pred[np.arange(n), labels])) / n

In [4]:
class NeuralNet:
    def __init__(self, input_size=784, hidden_size=128, output_size=10, lr=0.01):
        self.lr = lr
        
        # Xavier initialization
        self.W1 = np.random.randn(input_size, hidden_size) / np.sqrt(input_size)
        self.b1 = np.zeros((1, hidden_size))
        
        self.W2 = np.random.randn(hidden_size, output_size) / np.sqrt(hidden_size)
        self.b2 = np.zeros((1, output_size))

    def forward(self, x):
        self.x = x                     # (batch, 784)
        self.z1 = x @ self.W1 + self.b1
        self.a1 = relu(self.z1)
        self.z2 = self.a1 @ self.W2 + self.b2
        self.out = softmax(self.z2)
        return self.out

    def backward(self, labels):
        batch = labels.shape[0]

        # Output layer gradient
        dZ2 = self.out
        dZ2[np.arange(batch), labels] -= 1
        dZ2 /= batch
        
        # Gradients for W2, b2
        dW2 = self.a1.T @ dZ2
        db2 = np.sum(dZ2, axis=0, keepdims=True)

        # Backprop to hidden layer
        dA1 = dZ2 @ self.W2.T
        dZ1 = dA1 * relu_deriv(self.z1)

        dW1 = self.x.T @ dZ1
        db1 = np.sum(dZ1, axis=0, keepdims=True)

        # Gradient descent update
        self.W2 -= self.lr * dW2
        self.b2 -= self.lr * db2
        self.W1 -= self.lr * dW1
        self.b1 -= self.lr * db1

In [5]:
def train(model, x_train, y_train, epochs=5, batch_size=64):
    n = x_train.shape[0]
    for epoch in range(epochs):
        # shuffle dataset
        idx = np.random.permutation(n)
        x_train = x_train[idx]
        y_train = y_train[idx]

        losses = []

        for i in range(0, n, batch_size):
            xb = x_train[i:i+batch_size]
            yb = y_train[i:i+batch_size]

            pred = model.forward(xb)
            loss = cross_entropy(pred, yb)
            model.backward(yb)

            losses.append(loss)

        print(f"Epoch {epoch+1} Loss={np.mean(losses):.4f}")

In [6]:
def accuracy(model, x, y):
    preds = np.argmax(model.forward(x), axis=1)
    return np.mean(preds == y) * 100

In [7]:
# Load MNIST dataset
x_train = load_idx_images("train-images.idx3-ubyte")
y_train = load_idx_labels("train-labels.idx1-ubyte")
x_test  = load_idx_images("t10k-images.idx3-ubyte")
y_test  = load_idx_labels("t10k-labels.idx1-ubyte")

# Initialize network
model = NeuralNet(lr=0.01)

# Train
train(model, x_train, y_train, epochs=5, batch_size=128)

# Evaluate
print("Train accuracy:", accuracy(model, x_train, y_train))
print("Test accuracy:", accuracy(model, x_test, y_test))

Epoch 1 Loss=1.2914
Epoch 2 Loss=0.5977
Epoch 3 Loss=0.4641
Epoch 4 Loss=0.4079
Epoch 5 Loss=0.3755
Train accuracy: 89.95166666666667
Test accuracy: 90.42
